## How we can build a simple rnn network to have char-base autoencoder

In [717]:
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torch.functional import F
from torchinfo import summary

In [718]:
class cls_dataset(Dataset):
    def __init__(self, fix_len = 0) -> None:
        super().__init__()

        self.fix_len = fix_len

        with open("names.txt", "r") as f:
            s = f.read()

        self.m_names = s.split()

    def __len__(self):
        return len(self.m_names)


    def f_encode(self, str_t):
        
        t_res = torch.zeros((self.fix_len, 27),dtype=torch.int32)
        
        t_res[:,0] = 1
        for i in range(t_res.shape[0]):
            
            if i < len(str_t) :
                t_res[i][0] = 0
                t_res[i][ord(str_t[i])-96] = 1
            else:
                break

        return t_res


    def __getitem__(self, index) :
        x = self.f_encode(self.m_names[index])
        x = x.type(torch.float32)
        return x  , x

In [719]:
class cls_model(nn.Module):
    def __init__(self, num_layer = 2) -> None:
        super().__init__()

        self.m_layers = num_layer
        self.rnn1 = nn.RNN(input_size=27, hidden_size=27, num_layers= num_layer)
        self.hlayer = nn.Sequential( nn.Linear(27,27), nn.ReLU(), nn.Linear(27,27))
        self.rnn2 = nn.RNN(input_size=27, hidden_size=27, num_layers= num_layer)
        

    def forward(self, x):

        h0 = torch.zeros((self.m_layers, x.shape[1], self.rnn1.hidden_size ))
        x,_ = self.rnn1.forward(x, h0)
        x = self.hlayer.forward(x)
        x,_ = self.rnn2.forward(x, h0)

        return x

In [720]:
ds = cls_dataset(8)

ds_loader = DataLoader(ds, batch_size=16)

In [722]:
model = cls_model(2)

criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=3e-4)

epochs = 3
model.train()

for i in range(epochs):
    for idx, (x, y) in enumerate(ds_loader):
        
        y_pred = model.forward(x)
        
        optim.zero_grad()
        
        loss = criterion.forward(y_pred, y)

        loss.backward()
        optim.step()
            
    print(loss)

tensor(0.3750, grad_fn=<DivBackward1>)
tensor(0.3552, grad_fn=<DivBackward1>)
tensor(0.3513, grad_fn=<DivBackward1>)
tensor(0.3483, grad_fn=<DivBackward1>)


In [723]:
print(ds[100][0].argmax(dim=1))
model.forward(ds[100][0][None,:,:]).squeeze(dim=0).argmax(dim=1)

tensor([16,  5, 25, 20, 15, 14,  0,  0])


tensor([ 6,  5, 25, 20, 15, 14,  0,  0])

In [724]:
summary(model)

Layer (type:depth-idx)                   Param #
cls_model                                --
├─RNN: 1-1                               3,024
├─Sequential: 1-2                        --
│    └─Linear: 2-1                       756
│    └─ReLU: 2-2                         --
│    └─Linear: 2-3                       756
├─RNN: 1-3                               3,024
Total params: 7,560
Trainable params: 7,560
Non-trainable params: 0